In [1]:
# ==================================================
# 01_Feature_Engineering.ipynb
# Advanced Feature Engineering for Stock Prediction
# ==================================================

import pandas as pd
import numpy as np
import sqlite3
import warnings
warnings.filterwarnings('ignore')

print("AI Stock Management System - Feature Engineering")
print("=" * 60)

print("Step 1: Loading and Enhancing Data")
print("=" * 60)

def load_and_enhance_data():
    """Load existing data and add sophisticated features"""
    try:
        conn = sqlite3.connect("hybrid_ai_management_system_FIXED.db")
        df = pd.read_sql("SELECT * FROM hybrid_trading_data", conn)
        conn.close()
        
        df['date'] = pd.to_datetime(df['date'])
        print(f"Loaded existing data: {df.shape}")
        print(f"Date range: {df['date'].min().date()} to {df['date'].max().date()}")
        print(f"Symbols: {df['symbol'].unique().tolist()}")
        
        return df
    except Exception as e:
        print(f"Error loading data: {e}")
        return None

# Load data
df = load_and_enhance_data()

if df is None:
    print("No data available")
    exit()

# ==================================================
# Step 2: Advanced Feature Engineering
# ==================================================
print("\n" + "=" * 60)
print("Step 2: Advanced Feature Engineering")
print("=" * 60)

def create_advanced_features(df):
    """Create sophisticated trading features"""
    df_advanced = df.copy()
    df_advanced = df_advanced.sort_values(['symbol', 'date']).reset_index(drop=True)
    
    # 1. Price-based features
    print("Creating price-based features...")
    
    # A. Advanced Momentum
    df_advanced['momentum_acceleration'] = df_advanced.groupby('symbol')['return_5d'].diff()
    df_advanced['price_velocity'] = df_advanced['return_5d'] - df_advanced['return_10d']
    df_advanced['trend_persistence'] = (df_advanced['return_5d'] * df_advanced['return_10d'] > 0).astype(int)
    
    # B. Support/Resistance Levels
    for window in [10, 20, 50]:
        df_advanced[f'support_{window}d'] = df_advanced.groupby('symbol')['low'].rolling(window).min().reset_index(level=0, drop=True)
        df_advanced[f'resistance_{window}d'] = df_advanced.groupby('symbol')['high'].rolling(window).max().reset_index(level=0, drop=True)
        df_advanced[f'price_vs_support_{window}d'] = (df_advanced['close'] - df_advanced[f'support_{window}d']) / df_advanced[f'support_{window}d']
        df_advanced[f'price_vs_resistance_{window}d'] = (df_advanced[f'resistance_{window}d'] - df_advanced['close']) / df_advanced['close']
    
    # 2. Volume-based features
    print("Creating volume-based features...")
    
    # A. Volume Confirmation
    df_advanced['volume_trend'] = (df_advanced.groupby('symbol')['volume'].rolling(5).mean().reset_index(level=0, drop=True) > 
                                  df_advanced.groupby('symbol')['volume'].rolling(20).mean().reset_index(level=0, drop=True)).astype(int)
    df_advanced['volume_price_alignment'] = (df_advanced['daily_return'] * df_advanced['volume_zscore'] > 0).astype(int)
    
    # B. Smart Money Indicators
    df_advanced['large_volume_spike'] = (df_advanced['volume'] > df_advanced.groupby('symbol')['volume'].rolling(20).mean().reset_index(level=0, drop=True) * 2).astype(int)
    df_advanced['volume_climax'] = (df_advanced['volume'] > df_advanced.groupby('symbol')['volume'].rolling(50).quantile(0.9).reset_index(level=0, drop=True)).astype(int)
    
    # 3. Market regime features
    print("Creating market regime features...")
    
    # A. Volatility Regime
    df_advanced['volatility_regime'] = np.where(
        df_advanced['volatility_10d'] > df_advanced['volatility_10d'].quantile(0.7), 'HIGH',
        np.where(df_advanced['volatility_10d'] < df_advanced['volatility_10d'].quantile(0.3), 'LOW', 'MEDIUM')
    )
    
    # B. Trend Regime
    df_advanced['trend_regime'] = np.where(
        df_advanced['price_vs_MA_50'] > 0.05, 'STRONG_UP',
        np.where(df_advanced['price_vs_MA_50'] < -0.05, 'STRONG_DOWN', 'SIDEWAYS')
    )
    
    # C. Market Health (SPY trend)
    spy_data = df_advanced[df_advanced['symbol'] == 'SPY'][['date', 'close']].copy()
    spy_data['SPY_trend'] = (spy_data['close'] > spy_data['close'].rolling(50).mean()).astype(int)
    df_advanced = df_advanced.merge(spy_data[['date', 'SPY_trend']], on='date', how='left')
    
    # 4. Enhanced sentiment features
    print("Creating enhanced sentiment features...")
    
    # A. Contextual Sentiment
    df_advanced['sentiment_in_trend'] = (df_advanced['sentiment_with_momentum'] * df_advanced['return_5d'] > 0).astype(int)
    df_advanced['contrarian_sentiment'] = (df_advanced['sentiment_with_momentum'] * df_advanced['return_5d'] < 0).astype(int)
    
    # B. Sentiment Regimes
    df_advanced['sentiment_regime'] = np.where(
        df_advanced['sentiment_with_momentum'].abs() > 0.15, 'HIGH_SENTIMENT',
        np.where(df_advanced['sentiment_with_momentum'].abs() < 0.05, 'LOW_SENTIMENT', 'MODERATE_SENTIMENT')
    )
    
    # 5. Technical pattern features
    print("Creating technical pattern features...")
    
    # A. RSI Patterns
    df_advanced['RSI_divergence'] = (
        (df_advanced['RSI_14'] > df_advanced['RSI_14'].shift(5)) & 
        (df_advanced['close'] < df_advanced['close'].shift(5))
    ).astype(int)
    
    # B. Moving Average Patterns
    df_advanced['MA_alignment_strength'] = (
        (df_advanced['MA_5'] > df_advanced['MA_20']).astype(int) +
        (df_advanced['MA_20'] > df_advanced['MA_50']).astype(int) +
        (df_advanced['MA_50'] > df_advanced['MA_200']).astype(int)
    )
    
    # C. Breakout Detection
    df_advanced['breakout_20d_high'] = (df_advanced['close'] > df_advanced.groupby('symbol')['high'].rolling(20).max().shift(1).reset_index(level=0, drop=True)).astype(int)
    df_advanced['breakdown_20d_low'] = (df_advanced['close'] < df_advanced.groupby('symbol')['low'].rolling(20).min().shift(1).reset_index(level=0, drop=True)).astype(int)
    
    # 6. Risk-adjusted features
    print("Creating risk-adjusted features...")
    
    # A. Sharpe-like ratios
    df_advanced['volatility_5d'] = df_advanced.groupby('symbol')['daily_return'].rolling(5).std().reset_index(level=0, drop=True)
    df_advanced['return_to_volatility_5d'] = df_advanced['return_5d'] / (df_advanced['volatility_5d'] + 0.001)
    df_advanced['return_to_volatility_10d'] = df_advanced['return_10d'] / (df_advanced['volatility_10d'] + 0.001)
    
    # B. Drawdown Features
    df_advanced['rolling_max'] = df_advanced.groupby('symbol')['close'].rolling(20).max().reset_index(level=0, drop=True)
    df_advanced['current_drawdown'] = (df_advanced['close'] - df_advanced['rolling_max']) / df_advanced['rolling_max']
    
    # Fill NaN values
    numeric_cols = df_advanced.select_dtypes(include=[np.number]).columns
    df_advanced[numeric_cols] = df_advanced.groupby('symbol')[numeric_cols].ffill().fillna(0)
    
    print(f"Created {df_advanced.shape[1] - df.shape[1]} new advanced features")
    return df_advanced

# Apply advanced feature engineering
enhanced_df = create_advanced_features(df)
print(f"Enhanced dataset: {enhanced_df.shape}")

# ==================================================
# Step 3: Smart Target Engineering
# ==================================================
print("\n" + "=" * 60)
print("Step 3: Smart Target Engineering")
print("=" * 60)

def create_smart_targets(df):
    """Create multiple sophisticated trading targets"""
    df_targets = df.copy()
    
    # 1. Directional targets
    print("Creating directional targets...")
    
    # A. Basic Direction (rename to avoid conflicts)
    df_targets['target_direction_5d'] = (df_targets['future_return_5d'] > 0).astype(int)
    
    # B. Strong Move Target (filter out noise)
    strong_threshold = df_targets['future_return_5d'].abs().quantile(0.6)
    df_targets['target_strong_5d'] = (
        (df_targets['future_return_5d'] > strong_threshold) | 
        (df_targets['future_return_5d'] < -strong_threshold)
    ).astype(int)
    
    # 2. Magnitude targets (Regression)
    print("Creating magnitude targets...")
    
    df_targets['target_magnitude_5d'] = df_targets['future_return_5d'].abs()
    df_targets['target_high_vol_5d'] = (df_targets['target_magnitude_5d'] > 0.03).astype(int)
    
    # 3. Relative performance targets
    print("Creating relative performance targets...")
    
    # Beat SPY target
    spy_returns = df_targets[df_targets['symbol'] == 'SPY'][['date', 'future_return_5d']].rename(
        columns={'future_return_5d': 'SPY_future_return_5d'}
    )
    df_targets = df_targets.merge(spy_returns, on='date', how='left')
    df_targets['target_beat_spy_5d'] = (df_targets['future_return_5d'] > df_targets['SPY_future_return_5d']).astype(int)
    
    # 4. Risk-adjusted targets
    print("Creating risk-adjusted targets...")
    
    df_targets['target_sharpe_5d'] = df_targets['future_return_5d'] / (df_targets['volatility_5d'] + 0.001)
    df_targets['target_good_risk_5d'] = (df_targets['target_sharpe_5d'] > 1.0).astype(int)
    
    # 5. Regime-specific targets
    print("Creating regime-specific targets...")
    
    # High volatility regime target
    high_vol_mask = df_targets['volatility_regime'] == 'HIGH'
    df_targets['target_high_vol_success'] = ((df_targets['future_return_5d'].abs() > 0.02) & high_vol_mask).astype(int)
    
    print("Created multiple sophisticated targets")
    return df_targets

# Apply smart target engineering
targeted_df = create_smart_targets(enhanced_df)
print(f"Dataset with targets: {targeted_df.shape}")

# ==================================================
# Step 4: Feature Selection & Validation
# ==================================================
print("\n" + "=" * 60)
print("Step 4: Feature Selection & Validation")
print("=" * 60)

def select_best_features(df, target_column='target_direction_5d'):
    """Select most predictive features"""
    
    # Features to exclude (metadata, future data, targets, etc.)
    exclude_features = [
        'symbol', 'name', 'category', 'has_sentiment', 'date',
        'target_strong_5d', 'target_magnitude_5d', 'target_high_vol_5d', 
        'SPY_future_return_5d', 'target_beat_spy_5d', 'target_sharpe_5d', 
        'target_good_risk_5d', 'target_high_vol_success',
        'future_return_5d', 'future_return_10d', 'future_return_21d',
        'target_5d_binary', 'target_10d_binary', 'target_21d_binary',  # Remove old targets
        'target_5d_strong', 'target_10d_strong', 'target_21d_strong'   # Remove old targets
    ]
    
    # Get numeric features
    numeric_features = df.select_dtypes(include=[np.number]).columns
    candidate_features = [f for f in numeric_features if f not in exclude_features]
    
    # Calculate feature correlations with target
    correlations = []
    for feature in candidate_features:
        try:
            corr = abs(df[feature].corr(df[target_column]))
            if not np.isnan(corr):
                correlations.append((feature, corr))
        except:
            continue
    
    # Sort by correlation strength
    correlations.sort(key=lambda x: x[1], reverse=True)
    
    # Select top features (excluding the target itself)
    top_features = []
    for feature, corr in correlations:
        if feature != target_column and len(top_features) < 25:
            top_features.append(feature)
    
    print(f"Selected {len(top_features)} most predictive features")
    print(f"Top 10 features by correlation:")
    for feature, corr in correlations[:10]:
        if feature != target_column:
            print(f"   {feature:30}: {corr:.4f}")
    
    return top_features

# Select best features
best_features = select_best_features(targeted_df)
print(f"Total features available: {len(best_features)}")

# ==================================================
# Step 5: Final Dataset Preparation
# ==================================================
print("\n" + "=" * 60)
print("Step 5: Final Dataset Preparation")
print("=" * 60)

# Create final dataset - use unique target names
final_columns = ['symbol', 'name', 'category', 'date'] + best_features + [
    'target_direction_5d', 'target_strong_5d', 'target_magnitude_5d', 
    'target_beat_spy_5d', 'volatility_regime', 'trend_regime'
]

# Ensure all columns exist
available_columns = [col for col in final_columns if col in targeted_df.columns]
final_df = targeted_df[available_columns]

# Remove any remaining NaN values
final_df = final_df.dropna()
print(f"Final dataset: {final_df.shape}")

# Save enhanced dataset
conn = sqlite3.connect("enhanced_trading_dataset_v2.db")
final_df.to_sql("enhanced_trading_data", conn, if_exists="replace", index=False)

# Save feature metadata
feature_metadata = pd.DataFrame({
    'feature_name': best_features,
    'feature_type': 'ENHANCED_FEATURE',
    'category': 'TECHNICAL_INDICATOR'
})
feature_metadata.to_sql("feature_metadata", conn, if_exists="replace", index=False)

conn.close()

print("Enhanced dataset saved: enhanced_trading_dataset_v2.db")

# ==================================================
# Step 6: Dataset Analysis
# ==================================================
print("\n" + "=" * 60)
print("Step 6: Dataset Analysis")
print("=" * 60)

print("Target Distribution:")
print(f"   5-Day UP: {final_df['target_direction_5d'].mean():.1%}")
print(f"   Strong Moves: {final_df['target_strong_5d'].mean():.1%}")
print(f"   Beat SPY: {final_df['target_beat_spy_5d'].mean():.1%}")

print(f"\nFeature Categories:")
feature_categories = {
    'Momentum': len([f for f in best_features if 'momentum' in f or 'return' in f]),
    'Volatility': len([f for f in best_features if 'volatility' in f]),
    'Price_Level': len([f for f in best_features if 'price_vs' in f or 'support' in f or 'resistance' in f]),
    'Volume': len([f for f in best_features if 'volume' in f]),
    'Sentiment': len([f for f in best_features if 'sentiment' in f]),
    'Technical': len([f for f in best_features if 'RSI' in f or 'MA_' in f or 'break' in f])
}

for category, count in feature_categories.items():
    if count > 0:
        print(f"   {category:15}: {count} features")

print(f"\nSymbol Performance:")
for symbol in final_df['symbol'].unique():
    symbol_data = final_df[final_df['symbol'] == symbol]
    print(f"   {symbol}:")
    print(f"      Records: {len(symbol_data)}")
    print(f"      Win Rate: {symbol_data['target_direction_5d'].mean():.1%}")
    if 'target_beat_spy_5d' in symbol_data.columns:
        print(f"      Beat SPY: {symbol_data['target_beat_spy_5d'].mean():.1%}")

if 'volatility_regime' in final_df.columns:
    print(f"\nMarket Regime Distribution:")
    regime_counts = final_df['volatility_regime'].value_counts()
    for regime, count in regime_counts.items():
        print(f"   {regime:15}: {count} records ({count/len(final_df):.1%})")

print("\n" + "=" * 60)
print("Enhanced Data Pipeline Complete!")
print("=" * 60)

print(f"\nReady for Advanced Modeling:")
print(f"   Records: {len(final_df)}")
print(f"   Features: {len(best_features)}")
print(f"   Targets: Multiple trading strategies")
print(f"   Symbols: {final_df['symbol'].nunique()} diversified assets")
print(f"   Data Quality: Enhanced")

AI Stock Management System - Feature Engineering
Step 1: Loading and Enhancing Data
Loaded existing data: (2824, 49)
Date range: 2023-01-03 to 2025-10-24
Symbols: ['AAPL', 'AMZN', 'MSFT', 'SPY']

Step 2: Advanced Feature Engineering
Creating price-based features...
Creating volume-based features...
Creating market regime features...
Creating enhanced sentiment features...
Creating technical pattern features...
Creating risk-adjusted features...
Created 34 new advanced features
Enhanced dataset: (2824, 83)

Step 3: Smart Target Engineering
Creating directional targets...
Creating magnitude targets...
Creating relative performance targets...
Creating risk-adjusted targets...
Creating regime-specific targets...
Created multiple sophisticated targets
Dataset with targets: (2824, 92)

Step 4: Feature Selection & Validation
Selected 25 most predictive features
Top 10 features by correlation:
   price_vs_MA_200               : 0.0729
   trend_persistence             : 0.0624
   price_vs_suppo